In [ ]:
import re
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
path = r"C:\Users\RiL\Desktop\Prog\data"
dossier = os.listdir(path)
dossier

['accepted_2007_to_2018Q4.csv']

In [3]:
dossier_acc = path + dossier[0]
acc_fn = path + '\\' + dossier[0]
acc_fn
if os.path.isfile(acc_fn):
    print('all goods')
else:
    print('warning!')

all goods


In [4]:
# Chargement de la base de donnée
acc_df = pd.read_csv(acc_fn, low_memory=False)
acc_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Une boucle supprimant les variables qui n'ont aucune observations
for i in acc_df.columns:
    if acc_df[i].values.all() == True:
        acc_data = acc_df.drop([i], axis=1)

In [6]:
acc_data.shape
# Une seule variable supprimé !

(2260701, 150)

In [7]:
print(acc_data.shape)
acc_data.info()

(2260701, 150)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 150 entries, id to settlement_term
dtypes: float64(112), object(38)
memory usage: 2.5+ GB


In [8]:
# On va checker le pourcentage des NaN pour une variable particulière
j = acc_data['settlement_term'].isna().sum()
print((j / acc_data.shape[0])*100)

98.48516013395844


In [9]:
# j'exporte les noms des variables afin de les identifier et voir celles qui sont interessantes a analyser
var_names = pd.DataFrame(acc_data.columns)
var_names.to_excel('var_names.xlsx', index=False)
'''Voici le lien utilisé qui recense les definitions des variables et nous permettant d identifier les variables
liees a la contre-partie, ceux liees a la conjoncture economique si possible et d autres qui pourrait nous
interesse dans le cadre de l estimation de la fonction score'''
# https://rstudio-pubs-static.s3.amazonaws.com/290261_676d9bb194ae4c9882f599e7c0a808f2.html

'Voici le lien utilisé qui recense les definitions des variables et nous permettant d identifier les variables\nliees a la contre-partie, ceux liees a la conjoncture economique si possible et d autres qui pourrait nous\ninteresse dans le cadre de l estimation de la fonction score'

In [38]:
var_d_interet = ['term', 'int_rate', 'installment', 'grade', 'emp_length', 'home_ownership', 'loan_status', 'dti', 'revol_bal', 'total_acc', 'tot_cur_bal',
                 'open_acc_6m', 'total_bal_il', 'mort_acc', 'num_sats', 'out_prncp', 'total_pymnt', 'recoveries' ]

In [39]:
data = acc_data[var_d_interet]

In [49]:
data_final = data.loc[data['loan_status'].isin(['Charged Off', 'Fully Paid']), var_d_interet]
print(data_final.shape)
data_final.head()
data_final.to_csv('data.csv', header=True)


(1345310, 18)


In [59]:
data_final = pd.read_csv('data.csv')
data_final = data_final.drop(['Unnamed: 0'], axis=1)
print(data_final.shape)
data_final.head()

(1345310, 18)


,term,int_rate,installment,grade,emp_length,home_ownership,loan_status,dti,revol_bal,total_acc,tot_cur_bal,open_acc_6m,total_bal_il,mort_acc,num_sats,out_prncp,total_pymnt,recoveries
0,36 months,13.99,123.03,C,10+ years,MORTGAGE,Fully Paid,5.91,2765.0,13.0,144904.0,2.0,4981.0,1.0,7.0,0.0,4421.723917,0.0
1,36 months,11.99,820.28,C,10+ years,MORTGAGE,Fully Paid,16.06,21470.0,38.0,204396.0,1.0,18005.0,4.0,22.0,0.0,25679.660000,0.0
2,60 months,10.78,432.66,B,10+ years,MORTGAGE,Fully Paid,10.78,7869.0,18.0,189699.0,0.0,10827.0,5.0,6.0,0.0,22705.924294,0.0
3,60 months,22.45,289.91,F,3 years,MORTGAGE,Fully Paid,25.37,21929.0,35.0,331730.0,1.0,73839.0,6.0,12.0,0.0,11740.500000,0.0
4,36 months,13.44,405.18,C,4 years,RENT,Fully Paid,10.20,8822.0,6.0,12798.0,0.0,3976.0,0.0,5.0,0.0,13708.948530,0.0


In [60]:
data_final.isna().sum()

term                   0
int_rate               0
installment            0
grade                  0
emp_length         78511
home_ownership         0
loan_status            0
dti                  374
revol_bal              0
total_acc              0
tot_cur_bal        67527
open_acc_6m       807713
total_bal_il      807712
mort_acc           47281
num_sats           55841
out_prncp              0
total_pymnt            0
recoveries             0
dtype: int64

In [13]:
# Traitement des valeurs manquantes
'''Pour les variables qui ont un taux de non-reponse inferieur a 10% on va imputer par la moyenne ou le mode'''

'Pour les variables qui ont un taux de non-reponse inferieur a 10% on va imputer par la moyenne ou le mode'

In [14]:
def na_percent(data, var):
    per = round((data[var].isna().sum() / data.shape[0]) * 100, 2)
    return per

In [57]:
na_per = {col: na_percent(data_final, col) for col in var_d_interet}
col_fill = [p for p, v in na_per.items() if v <= 10 and v > 0]

for i in data_final[col_fill]:
    if data_final[i].dtypes == float:
        data_final[i] = data_final[i].fillna(data_final[i].mean())
    else:
        data_final[i] = data_final[i].fillna(data_final[i].mode()[0])   

In [59]:
data_final

,term,int_rate,installment,grade,emp_length,home_ownership,loan_status,dti,revol_bal,total_acc,tot_cur_bal,open_acc_6m,total_bal_il,mort_acc,num_sats
0,36 months,13.99,123.03,C,10+ years,MORTGAGE,Fully Paid,5.91,2765.0,13.0,144904.0,2.0,4981.0,1.0,7.0
1,36 months,11.99,820.28,C,10+ years,MORTGAGE,Fully Paid,16.06,21470.0,38.0,204396.0,1.0,18005.0,4.0,22.0
2,60 months,10.78,432.66,B,10+ years,MORTGAGE,Fully Paid,10.78,7869.0,18.0,189699.0,0.0,10827.0,5.0,6.0
4,60 months,22.45,289.91,F,3 years,MORTGAGE,Fully Paid,25.37,21929.0,35.0,331730.0,1.0,73839.0,6.0,12.0
5,36 months,13.44,405.18,C,4 years,RENT,Fully Paid,10.20,8822.0,6.0,12798.0,0.0,3976.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,60 months,9.49,377.95,B,5 years,OWN,Fully Paid,20.59,23833.0,39.0,168758.0,0.0,30516.0,3.0,17.0
2260690,60 months,13.99,683.94,C,9 years,MORTGAGE,Fully Paid,22.03,77480.0,32.0,663465.0,0.0,70652.0,3.0,16.0
2260691,60 months,14.49,752.74,C,3 years,MORTGAGE,Charged Off,10.34,111598.0,18.0,408701.0,2.0,11274.0,3.0,14.0
2260692,60 months,12.79,362.34,C,10+ years,RENT,Fully Paid,12.25,7700.0,28.0,65797.0,1.0,58097.0,0.0,12.0
